# Weaviate

- Author: [Haseom Shin](https://github.com/IHAGI-c)
- Design: []()
- Peer Review: []()
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/13-LangChain-Expression-Language/11-Fallbacks.ipynb) [![Open in GitHub](https://img.shields.io/badge/Open%20in%20GitHub-181717?style=flat-square&logo=github&logoColor=white)](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/13-LangChain-Expression-Language/11-Fallbacks.ipynb)

## Overview

This notebook covers how to get started with the Weaviate vector store in LangChain, using the `langchain-weaviate` package.

> [Weaviate](https://weaviate.io/) is an open-source vector database. It allows you to store data objects and vector embeddings from your favorite ML-models, and scale seamlessly into billions of data objects.

To use this integration, you need to have a running Weaviate database instance.

### Table of Contents

- [Overview](#overview)
- [Environment Setup](#environment-setup)
- [Credentials](#credentials)
  - [Setting up Weaviate Cloud Services](#setting-up-weaviate-cloud-services)
- [What is Weaviate?](#what-is-weaviate)
- [Why Use Weaviate?](#why-use-weaviate)
- [Initialization](#initialization)
  - [List Indexs](#list-indexs)
  - [Create Index](#create-index)
  - [Delete Index](#delete-index)
  - [Select Embeddings model](#select-embeddings-model)
  - [Data Preprocessing](#data-preprocessing)
- [Manage vector store](#manage-vector-store)
  - [Add items to vector store](#add-items-to-vector-store)
  - [Delete items from vector store](#delete-items-from-vector-store)
- [Finding Objects by Similarity](#finding-objects-by-similarity)
  - [Step 1: Preparing Your Data](#step-1-preparing-your-data)
  - [Step 2: Perform the search](#step-2-perform-the-search)
  - [Quantify Result Similarity](#quantify-result-similarity)
- [Search mechanism](#search-mechanism)
- [Persistence](#persistence)
- [Multi-tenancy](#multi-tenancy)
- [Retriever options](#retriever-options)
- [Use with LangChain](#use-with-langchain)
  - [Question Answering with Sources](#question-answering-with-sources)
  - [Retrieval-Augmented Generation](#retrieval-augmented-generation)


### References
- [Langchain-Weaviate](https://python.langchain.com/docs/integrations/providers/weaviate/)
- [Weaviate Documentation](https://weaviate.io/developers/weaviate)
- [Weaviate Introduction](https://weaviate.io/developers/weaviate/introduction)
---

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- `langchain-opentutorial` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [`langchain-opentutorial`](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [1]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [2]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "openai",
        "langsmith",
        "langchain",
        "tiktoken",
        "langchain-weaviate",
        "langchain-openai",
    ],
    verbose=False,
    upgrade=False,
)


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "WEAVIATE_API_KEY": "",
        "WEAVIATE_URL": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "Weaviate",
    }
)

Environment variables have been set successfully.


You can alternatively set `OPENAI_API_KEY` in `.env` file and load it. 

[Note] This is not necessary if you've already set `OPENAI_API_KEY` in previous steps.

In [4]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

## Credentials

There are three main ways to connect to Weaviate:

1. **Local Connection**: Connect to a Weaviate instance running locally through Docker
2. **Weaviate Cloud Services (WCS)**: Use Weaviate's managed cloud service
3. **Custom Deployment**: Deploy Weaviate on Kubernetes or other custom configurations

For this notebook, we'll use Weaviate Cloud Services (WCS) as it provides the easiest way to get started without any local setup.

### Setting up Weaviate Cloud Services

1. First, sign up for a free account at [Weaviate Cloud Console](https://console.weaviate.cloud)
2. Create a new cluster
3. Get your API key
4. Set API key
5. Connect to your WCS cluster

#### 1. Weaviate Signup
![Weaviate Cloud Console](./assets/09-Weaviate-Credentials-01.png)

#### 2. Create Cluster
![Weaviate Cloud Console](./assets/09-Weaviate-Credentials-02.png)
![Weaviate Cloud Console](./assets/09-Weaviate-Credentials-03.png)

#### 3. Get API Key
**If you using gRPC, please copy the gRPC URL**

![Weaviate Cloud Console](./assets/09-Weaviate-Credentials-04-1.png)

#### 4. Set API Key
```
WEAVIATE_API_KEY="YOUR_WEAVIATE_API_KEY"
WEAVIATE_URL="YOUR_WEAVIATE_CLUSTER_URL"
```

#### 5. Connect to your WCS cluster

In [5]:
import getpass
import os
import weaviate
from weaviate.classes.init import Auth

if not os.getenv("WEAVIATE_API_KEY"):
    os.environ["WEAVIATE_API_KEY"] = getpass.getpass("Enter your Weaviate API key: ")

if not os.getenv("WEAVIATE_URL"):
    os.environ["WEAVIATE_URL"] = getpass.getpass("Enter your Weaviate URL: ")

weaviate_url = os.environ.get("WEAVIATE_URL")
weaviate_api_key = os.environ.get("WEAVIATE_API_KEY")

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,
    auth_credentials=Auth.api_key(weaviate_api_key),
)

print(client.is_ready())

True


In [ ]:
## api key Lookup
def get_api_key():
    return weaviate_api_key

print(get_api_key())

## What is Weaviate?

Weaviate is a powerful open-source vector database that revolutionizes how we store and search data. It combines traditional database capabilities with advanced machine learning features, allowing you to:

- Weaviate is an open source [vector database](https://weaviate.io/blog/what-is-a-vector-database).
- Weaviate allows you to store and retrieve data objects based on their semantic properties by indexing them with [vectors](./concepts/vector-index.md).
- Weaviate can be used stand-alone (aka _bring your vectors_) or with a variety of [modules](./modules/index.md) that can do the vectorization for you and extend the core capabilities.
- Weaviate has a [GraphQL-API](./api/graphql/index.md) to access your data easily.
- Weaviate is fast (check our [open source benchmarks](./benchmarks/index.md)).

> 💡 **Key Feature**: Weaviate achieves millisecond-level query performance, making it suitable for production environments.

## Why Use Weaviate?

Weaviate stands out for several reasons:

1. **Versatility**: Supports multiple media types (text, images, etc.)
2. **Advanced Features**:
   - Semantic Search
   - Question-Answer Extraction
   - Classification
   - Custom ML Model Integration
3. **Production-Ready**: Built in Go for high performance and scalability
4. **Developer-Friendly**: Multiple access methods through GraphQL, REST, and various client libraries


## Initialization
Before initializing our vector store, let's connect to a Weaviate collection. If one named index_name doesn't exist, it will be created.

### List Indexs

In [7]:
def list_indexs():
    list_all = client.collections.list_all()
    existing_indexes = [collection.name for collection in list_all.values()]
    return existing_indexes

existing_indexes = list_indexs()
print(existing_indexes)

['LangChain_2c4c3edbbfee4d26b28fdb9e98be422a', 'LangChain_11e7e8895ea948988f248a9166a815c1', 'LangChain_036a326407ab421883034bff005ad58e', 'LangChain_1b7a38d8adbb4847920f6f6fbf1015a3', 'LangChain_3cfe15e2532249188454455d7f9fcf47', 'LangChain_ab1fd6a408854159a985b2a2f2ca1bc7', 'LangChain_99833dcb8b534a83b3eda9325db5a064', 'LangChain_591a2927d8ef495f8fb8719f8e680622', 'LangChain_1fa3480e8957497fa001e0e22d629d8e', 'LangChain_579be4b8513a48788679bc1e6ec4eefe', 'LangChain_dd785d126146448e80ff55e3d3957fac', 'LangChain_d92aa0dd43a14214b299c91345c8d427', 'LangChain_724554a33196495cae3e4129f5ad697b', 'LangChain_727079168b704c33bdf10725510df362', 'LangChain_a481c431386e4385836736688caf6594', 'LangChain_dc99b66aa570435fb528a7dda11f2c57', 'LangChain_f3adf1f1210c429ca1944d9259a2d15d', 'LangChain_00a20c1453ad46f58101f9c880b4df55', 'LangChain_519b7652237948578e4a04ebc83b9096', 'LangChain_86ebcc56a5354505845a02aca082481c', 'LangChain_36336a662af7405b89d8d1e60ab90c5c', 'LangChain_20704abf874e49c8802a70

In [8]:
index_name = "Langchain_test_index6"

def lookup_index(index_name: str):
    return client.collections.get(index_name)

print(lookup_index(index_name))

<weaviate.Collection config={
  "name": "Langchain_test_index6",
  "description": null,
  "generative_config": null,
  "inverted_index_config": {
    "bm25": {
      "b": 0.75,
      "k1": 1.2
    },
    "cleanup_interval_seconds": 60,
    "index_null_state": false,
    "index_property_length": false,
    "index_timestamps": false,
    "stopwords": {
      "preset": "en",
      "additions": null,
      "removals": null
    }
  },
  "multi_tenancy_config": {
    "enabled": false,
    "auto_tenant_creation": false,
    "auto_tenant_activation": false
  },
  "properties": [
    {
      "name": "content",
      "description": null,
      "data_type": "text",
      "index_filterable": true,
      "index_range_filters": false,
      "index_searchable": true,
      "nested_properties": null,
      "tokenization": "word",
      "vectorizer_config": {
        "skip": false,
        "vectorize_property_name": true
      },
      "vectorizer": "text2vec-openai"
    },
    {
      "name": "metadat

### Create Index

Creates a new index in Weaviate.



In [ ]:
from typing import Any, Optional, Dict
from weaviate.classes.config import Property, DataType, Configure
from weaviate.classes.config import Configure, VectorDistances
from weaviate import WeaviateClient

properties_names = ["chapter", "index", "total_docs"]

def create_index(
    client: WeaviateClient,
    index_name: str,
    dimension: int,
    metric: str = "cosine",
    pod_spec: Optional[Dict] = None,
    **kwargs
) -> Any:
    """
    Weaviate collection creation function
    """
    # Set distance_metric to VectorDistances enumeration
    distance_metric = getattr(VectorDistances, metric.upper(), None)

    # Set vector_index_config to hnsw
    vector_index_config = Configure.VectorIndex.hnsw(
        distance_metric=distance_metric
    )

    # Create property list
    properties = [
        Property(name="content", data_type=DataType.TEXT, index_filterable=True, index_searchable=True),
    ]
    
    # Add predefined properties
    for prop_name in properties_names:
        if prop_name in ["index", "total_docs"]:
            # Numeric field
            properties.append(
                Property(name=prop_name, data_type=DataType.INT, index_filterable=True)
            )
        else:
            # Text field
            properties.append(
                Property(name=prop_name, data_type=DataType.TEXT, index_filterable=True, index_searchable=True)
            )

    # Create collection
    collection = client.collections.create(
        name=index_name,
        vectorizer_config=Configure.Vectorizer.text2vec_openai(),
        properties=properties,
        vector_index_config=vector_index_config,
    )

    print(f"[Weaviate] Collection '{index_name}' created successfully")
    return collection

index_name = "Langchain_test_index"  # change if desired

if index_name not in existing_indexes:
    index = create_index(client=client, index_name=index_name, dimension=128, metric="dot")
    print(f"Created index: {index}")
else:
    print(f"Collection '{index_name}' already exists")

### Delete Index

In [ ]:
def delete_index(index_name: str):
    client.collections.delete(index_name)
    print(f"Deleted index: {index_name}")

delete_index(index_name)

### Select Embeddings model

In [13]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [14]:
from langchain_weaviate.vectorstores import WeaviateVectorStore

vector_store = WeaviateVectorStore(client=client, index_name=index_name, text_key="content", embedding=embeddings)

### Data Preprocessing

Below is the preprocessing process for general documents.

In [15]:
# This is a long document we can split up.
with open("./data/the_little_prince.txt") as f:
    raw_text = f.read()

In [20]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=200,
    chunk_overlap=30,
    length_function=len,
    is_separator_regex=False,
)

split_docs = text_splitter.create_documents([raw_text])

print(split_docs[:20])

[Document(metadata={}, page_content='The Little Prince\nWritten By Antoine de Saiot-Exupery (1900〜1944)'), Document(metadata={}, page_content='[ Antoine de Saiot-Exupery ]'), Document(metadata={}, page_content='Over the past century, the thrill of flying has inspired some to perform remarkable feats of daring. For others, their desire to soar into the skies led to dramatic leaps in technology. For Antoine'), Document(metadata={}, page_content='in technology. For Antoine de Saint-Exupéry, his love of aviation inspired stories, which have touched the hearts of millions around the world.'), Document(metadata={}, page_content='Born in 1900 in Lyons, France, young Antoine was filled with a passion for adventure. When he failed an entrance exam for the Naval Academy, his interest in aviation took hold. He joined the French'), Document(metadata={}, page_content='hold. He joined the French Army Air Force in 1921 where he first learned to fly a plane. Five years later, he would leave the milita

In [21]:
from typing import List, Dict, Optional
import re
from langchain_core.documents import Document

from typing import List, Dict, Optional, Tuple
import re

def preprocess_documents(
    split_docs: List[Document],
    metadata_keys: Optional[List[str]] = None,
    min_length: int = 5,
    use_basename: bool = True
) -> Tuple[List[str], List[Dict]]:
    processed_docs = []
    
    # Default metadata keys
    if metadata_keys is None:
        metadata_keys = ["source", "author", "chapter"]
    
    # Group documents by chapter
    chapter_pattern = re.compile(r'Chapter (\d+)')
    current_chapter = "Chapter 1"
    chapter_docs = {}
    
    # Group documents by chapter
    for doc in split_docs:
        if len(doc.page_content.strip()) < min_length:
            continue
            
        # Extract chapter information from the content
        chapter_match = chapter_pattern.search(doc.page_content)
        if chapter_match:
            current_chapter = f"Chapter {chapter_match.group(1)}"
        
        if current_chapter not in chapter_docs:
            chapter_docs[current_chapter] = []
        chapter_docs[current_chapter].append(doc)
    
    # Process documents for each chapter
    for chapter, docs in chapter_docs.items():
        for doc_idx, doc in enumerate(docs):
            # Create basic metadata
            metadata = {
                "chapter": chapter,
                "index": doc_idx,
                "total_docs": len(docs),
            }
            
            # Process existing metadata
            if hasattr(doc, 'metadata'):
                for key in metadata_keys:
                    if key in doc.metadata:
                        value = doc.metadata[key]
                        if key == "source" and use_basename:
                            from pathlib import Path
                            value = Path(value).name
                        metadata[key] = value
            
            # Create a new Document object
            processed_doc = Document(
                page_content=doc.page_content,
                metadata=metadata
            )
            processed_docs.append(processed_doc)
    
    return processed_docs, metadata_keys


processed_docs, metadata_keys = preprocess_documents(
    split_docs=split_docs,
    metadata_keys=properties_names,
    min_length=5,
    use_basename=True,
)

## Manage vector store
Once you have created your vector store, we can interact with it by adding and deleting different items.

### Add items to vector store

Weaviate supports dynamic batch processing, which allows you to add documents in parallel. This is useful when you have a large number of documents to add.

In [22]:
from typing import List

def upsert_documents(docs: List[Document]):
  return vector_store.add_documents(docs)

upsert_documents(docs=processed_docs)

['3613b60f-7367-4249-880a-22c11c1ee6e4',
 '6c26ded8-bac5-467b-aa80-5d04985dd1c7',
 '5a03beda-2f5b-4638-9f2b-0103612094ad',
 '848df8b7-d675-4949-89d9-e2b2c84f32bd',
 '5c2a27be-6667-4ca0-acbd-7ec9e93dd30e',
 '4691957b-72a2-4df7-8f57-21a5815a0aab',
 'bc3b429d-af3d-4632-acc9-7355f15bb47a',
 '4823af34-8854-4d4c-ba8e-13980147652e',
 '2b6dea1c-72e2-46ae-b000-02726e2eb7ca',
 'd73992cc-ac70-4a4d-813e-1e9051de800f',
 '888e7819-b023-4d29-a2b0-a39a56b2f4c6',
 '58ac61ae-e38b-4f74-8352-ee896e3e4de7',
 '71c133f6-9593-4b7e-bf7d-95294886daa8',
 '98a0edb5-e775-45f9-b966-abe6b684535d',
 'a12907ee-50b3-48bb-8d18-2159c0514eae',
 'faf4a09e-18b2-47b4-8b5c-3aae4b651f84',
 '831f4e51-f5fe-438c-a05b-06eccd9a60bd',
 'f2d30de6-0ee6-4127-8c59-c73afa86c85b',
 'd120bf58-dd84-4ceb-be8a-63de47345b58',
 '168ff878-8445-402a-847c-87a44a4ed7e1',
 '02c11e6e-de5c-4a6d-b5b5-2ff1d89af802',
 'c0066561-1e94-4e65-8b4f-7b2cb594af22',
 '01a47ea9-6af4-44f5-8667-6e9c27783ccd',
 '6833b9ae-15c3-46c4-b790-74a892afee3a',
 'b701c74f-bebe-

In [23]:
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

def upsert_documents_parallel(
    docs: List[Document],
    batch_size: int = 100,
    max_workers: Optional[int] = None,
    show_progress: bool = True
) -> List[str]:
    batches = [docs[i:i + batch_size] for i in range(0, len(docs), batch_size)]
    
    if show_progress:
        print(f"Total documents: {len(docs)}, Number of batches: {len(batches)}")
    
    def process_batch(batch: List[Document]) -> Optional[List[str]]:
        try:
            return vector_store.add_documents(batch)
        except Exception as e:
            print(f"Error occurred during batch processing: {e}")
            return None
    
    all_ids = []
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_batch = {
            executor.submit(process_batch, batch): i 
            for i, batch in enumerate(batches)
        }
        
        if show_progress:
            for future in tqdm(as_completed(future_to_batch), total=len(batches), desc="Uploading documents"):
                batch_result = future.result()
                if batch_result:
                    all_ids.extend(batch_result)
        else:
            for future in as_completed(future_to_batch):
                batch_result = future.result()
                if batch_result:
                    all_ids.extend(batch_result)
    
    return all_ids

start_time = time.time()

results = upsert_documents_parallel(
    docs=processed_docs,
    batch_size=100,
    max_workers=4,
    show_progress=True
)

end_time = time.time()

print(f"Upsert completed")
print(f"Processed documents: {len(results)}")
print(f"Time taken: {end_time - start_time:.2f} seconds")

Total documents: 698, Number of batches: 7


Uploading documents: 100%|██████████| 7/7 [00:06<00:00,  1.13it/s]

Upsert completed
Processed documents: 698
Time taken: 6.36 seconds


In [24]:
from weaviate.collections.classes.filters import Filter

filter_query = Filter.by_property("chapter").equal("Chapter 21")

vector_store.similarity_search(
    query="Who is the narrator of the story?",
    k=3,
    filters=filter_query
)

[Document(metadata={'chapter': 'Chapter 21', 'text': None, 'index': 2.0, 'total_docs': 52.0}, page_content='(picture)\n"Who are you?" asked the little prince, and added, "You are very pretty to look at." \n"I am a fox," said the fox. \n"Come and play with me," proposed the little prince. "I am so unhappy."'),
 Document(metadata={'chapter': 'Chapter 21', 'text': None, 'index': 2.0, 'total_docs': 52.0}, page_content='(picture)\n"Who are you?" asked the little prince, and added, "You are very pretty to look at." \n"I am a fox," said the fox. \n"Come and play with me," proposed the little prince. "I am so unhappy."'),
 Document(metadata={'chapter': 'Chapter 21', 'text': None, 'index': 1.0, 'total_docs': 52.0}, page_content='"Good morning," the little prince responded politely, although when he turned around he saw nothing. \n"I am right here," the voice said, "under the apple tree." \n(picture)')]

### Delete items from vector store

You can delete items from vector store by filter

In [25]:
from weaviate.collections.classes.filters import Filter
from typing import Optional

def delete_by_filter(filter_query: Filter) -> int:
    try:
        # Retrieve the collection
        collection = client.collections.get(index_name)
        
        # Check the number of documents that match the filter before deletion
        query_result = collection.query.fetch_objects(
            filters=filter_query,
        )
        initial_count = len(query_result.objects)
        
        # Delete documents that match the filter condition
        result = collection.data.delete_many(
            where=filter_query
        )
        
        print(f"Number of documents deleted: {initial_count}")
        return initial_count
        
    except Exception as e:
        print(f"Error occurred during deletion: {e}")
        raise
    
delete_by_filter(filter_query)

Number of documents deleted: 10


10

## Finding Objects by Similarity

Weaviate allows you to find objects that are semantically similar to your query. Let's walk through a complete example, from importing data to executing similarity searches.

### Step 1: Preparing Your Data

Before we can perform similarity searches, we need to populate our Weaviate instance with data. We'll start by loading and chunking a text file into manageable pieces.

> 💡 **Tip**: Breaking down large texts into smaller chunks helps optimize vector search performance and relevance.

In [26]:
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_text_splitters import CharacterTextSplitter
from langchain_weaviate.vectorstores import WeaviateVectorStore

# Create a document with metadata, including geo-information
raw_texts = [
    "The Eiffel Tower in Paris stands 324 meters tall and was completed in 1889.",
    "The Great Wall of China is over 21,000 kilometers long and was built over several centuries.",
    "The Taj Mahal in India was built by Emperor Shah Jahan as a tomb for his beloved wife.",
    "Machu Picchu in Peru was built by the Inca Empire in the 15th century at an altitude of 2,430 meters.",
    "The Pyramids of Giza in Egypt were built over 4,500 years ago as tombs for pharaohs.",
    "The Colosseum in Rome could hold up to 50,000 spectators for gladiatorial contests.",
    "Petra in Jordan was carved into rose-colored rock faces and served as a trading center.",
    "Angkor Wat in Cambodia is the world's largest religious monument, built in the 12th century."
]

# Regional information for each text
regions = [
    "Europe",    # Eiffel Tower
    "Asia",      # Great Wall
    "Asia",      # Taj Mahal
    "South America",  # Machu Picchu
    "Africa",    # Pyramids
    "Europe",    # Colosseum
    "Asia",      # Petra
    "Asia"       # Angkor Wat
]

docs = [
    Document(page_content=text, metadata={"region": region}) 
    for text, region in zip(raw_texts, regions)
]

embeddings = OpenAIEmbeddings()

db = WeaviateVectorStore.from_documents(docs, embeddings, client=client)

### Step 2: Perform the search

We can now perform a similarity search. This will return the most similar documents to the query text, based on the embeddings stored in Weaviate and an equivalent embedding generated from the query text.

In [27]:
query = "What is Petra?"
docs = db.similarity_search(query, k=1)

for i, doc in enumerate(docs):
    print(f"\nDocument {i+1}:")
    print(doc.page_content)


Document 1:
Petra in Jordan was carved into rose-colored rock faces and served as a trading center.


You can also add filters, which will either include or exclude results based on the filter conditions. (See [more filter examples](https://weaviate.io/developers/weaviate/search/filters).)

In [28]:
from weaviate.classes.query import Filter

for region in regions:
    search_filter = Filter.by_property("region").equal(region)
    filtered_results = db.similarity_search(query, filters=search_filter, k=4)
    
    print(f"\n=== Monuments in {region} ===")
    print(f"Found {len(filtered_results)} results:")
    for i, doc in enumerate(filtered_results, 1):
        print(f"\nDocument {i}:")
        print(f"Content: {doc.page_content}")
        print(f"Region: {doc.metadata['region']}")


=== Monuments in Europe ===
Found 2 results:

Document 1:
Content: The Colosseum in Rome could hold up to 50,000 spectators for gladiatorial contests.
Region: Europe

Document 2:
Content: The Eiffel Tower in Paris stands 324 meters tall and was completed in 1889.
Region: Europe

=== Monuments in Asia ===
Found 4 results:

Document 1:
Content: Petra in Jordan was carved into rose-colored rock faces and served as a trading center.
Region: Asia

Document 2:
Content: Angkor Wat in Cambodia is the world's largest religious monument, built in the 12th century.
Region: Asia

Document 3:
Content: The Taj Mahal in India was built by Emperor Shah Jahan as a tomb for his beloved wife.
Region: Asia

Document 4:
Content: The Great Wall of China is over 21,000 kilometers long and was built over several centuries.
Region: Asia

=== Monuments in Asia ===
Found 4 results:

Document 1:
Content: Petra in Jordan was carved into rose-colored rock faces and served as a trading center.
Region: Asia

Documen

It is also possible to provide `k`, which is the upper limit of the number of results to return.

In [29]:
# Using the k parameter to limit the number of results
search_filter = Filter.by_property("region").equal(regions[0])  # Europe
filtered_search_results = db.similarity_search(query, filters=search_filter, k=3)

print("\n=== Limiting Results with k parameter ===")
print(f"\nSearching for monuments in {regions[0]} with k=3:")
print(f"Number of results: {len(filtered_search_results)}")

for i, doc in enumerate(filtered_search_results, 1):
    print(f"\nResult {i}:")
    print(f"Content: {doc.page_content}")

# Check if the number of results is k or less
assert len(filtered_search_results) <= 3, f"Expected 3 or fewer results, but got {len(filtered_search_results)}"
print("\nVerification: ✓ Number of results is correctly limited by k parameter")


=== Limiting Results with k parameter ===

Searching for monuments in Europe with k=3:
Number of results: 2

Result 1:
Content: The Colosseum in Rome could hold up to 50,000 spectators for gladiatorial contests.

Result 2:
Content: The Eiffel Tower in Paris stands 324 meters tall and was completed in 1889.

Verification: ✓ Number of results is correctly limited by k parameter


### Quantify Result Similarity

When performing similarity searches, you might want to know not just which documents are similar, but how similar they are. Weaviate provides this information through a relevance score.
> 💡 Tip: The relevance score helps you understand the relative similarity between search results.

In [30]:
docs = db.similarity_search_with_score("What monuments are in Asia?", k=5)

for doc in docs:
    print(f"{doc[1]:.3f}", ":", doc[0].page_content)

1.000 : Angkor Wat in Cambodia is the world's largest religious monument, built in the 12th century.
0.728 : The Taj Mahal in India was built by Emperor Shah Jahan as a tomb for his beloved wife.
0.527 : Petra in Jordan was carved into rose-colored rock faces and served as a trading center.
0.509 : The Great Wall of China is over 21,000 kilometers long and was built over several centuries.
0.304 : The Pyramids of Giza in Egypt were built over 4,500 years ago as tombs for pharaohs.


## Search mechanism

`similarity_search` uses Weaviate's [hybrid search](https://weaviate.io/developers/weaviate/api/graphql/search-operators#hybrid).

A hybrid search combines a vector and a keyword search, with `alpha` as the weight of the vector search. The `similarity_search` function allows you to pass additional arguments as kwargs. See this [reference doc](https://weaviate.io/developers/weaviate/api/graphql/search-operators#hybrid) for the available arguments.

So, you can perform a pure keyword search by adding `alpha=0` as shown below:

In [31]:
docs = db.similarity_search(query, alpha=0)
docs[0]

Document(metadata={'region': 'Asia'}, page_content='Petra in Jordan was carved into rose-colored rock faces and served as a trading center.')

## Persistence

Any data added through `langchain-weaviate` will persist in Weaviate according to its configuration. 

WCS instances, for example, are configured to persist data indefinitely, and Docker instances can be set up to persist data in a volume. Read more about [Weaviate's persistence](https://weaviate.io/developers/weaviate/configuration/persistence).

## Multi-tenancy

[Multi-tenancy](https://weaviate.io/developers/weaviate/concepts/data#multi-tenancy) allows you to have a high number of isolated collections of data, with the same collection configuration, in a single Weaviate instance. This is great for multi-user environments such as building a SaaS app, where each end user will have their own isolated data collection.

To use multi-tenancy, the vector store need to be aware of the `tenant` parameter. 

So when adding any data, provide the `tenant` parameter as shown below.

In [32]:
# 2. Create a vector store with a specific tenant
db_with_tenant = WeaviateVectorStore.from_documents(
    docs, 
    embeddings, 
    client=client,
    tenant="tenant1"  # specify the tenant name
)


2025-Jan-18 10:05 PM - langchain_weaviate.vectorstores - INFO - Tenant tenant1 does not exist in index LangChain_572dd1ab2bcf4d7fba35f54699be1041. Creating tenant.


In [33]:
results = db_with_tenant.similarity_search(
    "What is Petra?",
    tenant="tenant1"  # use the same tenant name
)

for doc in results:
    print(doc.page_content)


Petra in Jordan was carved into rose-colored rock faces and served as a trading center.


In [34]:
db_with_mt = WeaviateVectorStore.from_documents(
    docs, embeddings, client=client, tenant="tenant1"
)

2025-Jan-18 10:05 PM - langchain_weaviate.vectorstores - INFO - Tenant tenant1 does not exist in index LangChain_c936ea9a49e64e5ea9edcc7b7a67b019. Creating tenant.


And when performing queries, provide the `tenant` parameter also.

In [35]:
db_with_mt.similarity_search(query, tenant="tenant1")

[Document(metadata={'region': 'Asia'}, page_content='Petra in Jordan was carved into rose-colored rock faces and served as a trading center.')]

## Retriever options

Weaviate can also be used as a retriever

### Maximal marginal relevance search (MMR)

In addition to using similaritysearch  in the retriever object, you can also use `mmr`.

In [36]:
retriever = db.as_retriever(search_type="mmr")
retriever.invoke(query)[0]

Document(metadata={'region': 'Asia'}, page_content='Petra in Jordan was carved into rose-colored rock faces and served as a trading center.')

## Use with LangChain

A known limitation of large language models (LLMs) is that their training data can be outdated, or not include the specific domain knowledge that you require.

Take a look at the example below:

In [37]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
result = llm.invoke("What is Eiffel Tower?")
print(result.content)

The Eiffel Tower is a wrought-iron lattice tower located on the Champ de Mars in Paris, France. It was designed by the engineer Gustave Eiffel and completed in 1889 as the entrance arch for the 1889 Exposition Universelle (World's Fair), which was held to celebrate the 100th anniversary of the French Revolution. 

Standing at approximately 300 meters (984 feet) tall, the Eiffel Tower was the tallest man-made structure in the world until the completion of the Chrysler Building in New York City in 1930. It is one of the most recognizable structures in the world and a global cultural icon of France.

The tower has three levels accessible to the public, with restaurants on the first and second levels and an observation deck on the third level, offering panoramic views of Paris. The Eiffel Tower is also known for its nightly illuminations and has become a symbol of romance and tourism in the city. It attracts millions of visitors each year, making it one of the most visited paid monuments i

Vector stores complement LLMs by providing a way to store and retrieve relevant information. This allow you to combine the strengths of LLMs and vector stores, by using LLM's reasoning and linguistic capabilities with vector stores' ability to retrieve relevant information.

Two well-known applications for combining LLMs and vector stores are:
- Question answering
- Retrieval-augmented generation (RAG)

### Question Answering with Sources

Question answering in langchain can be enhanced by the use of vector stores. Let's see how this can be done.

This section uses the `RetrievalQAWithSourcesChain`, which does the lookup of the documents from an Index. 

First, we will chunk the text again and import them into the Weaviate vector store.

In [38]:
docsearch = WeaviateVectorStore.from_texts(
    raw_texts,
    embeddings,
    client=client,
    metadatas=[{"source": f"{i}-pl"} for i in range(len(raw_texts))],
)

Now we can construct the chain, with the retriever specified:

In [39]:
from langchain.chains import RetrievalQAWithSourcesChain

chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [40]:
chain.invoke(
    {"question": "What is Eiffel Tower?"},
    return_only_outputs=True,
)

{'answer': 'The Eiffel Tower is a landmark in Paris that stands 324 meters tall and was completed in 1889.  \n',
 'sources': '0-pl'}

### Retrieval-Augmented Generation

Another very popular application of combining LLMs and vector stores is retrieval-augmented generation (RAG). This is a technique that uses a retriever to find relevant information from a vector store, and then uses an LLM to provide an output based on the retrieved data and a prompt.

We begin with a similar setup:

In [41]:
docsearch = WeaviateVectorStore.from_texts(
    raw_texts,
    embeddings,
    client=client,
    metadatas=[{"source": f"{i}-pl"} for i in range(len(raw_texts))],
)

retriever = docsearch.as_retriever()

We need to construct a template for the RAG model so that the retrieved information will be populated in the template.

In [42]:
from langchain_core.prompts import ChatPromptTemplate

template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

print(prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:\n"), additional_kwargs={})]


In [43]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Petra?")

'Petra is an archaeological site in Jordan, known for its stunning architecture carved into rose-colored rock faces. It served as a significant trading center in ancient times.'